_________________________________________________________________________

# **Data Scientist Test Task [v2.0] inPowered** 


Optimizing CPE across groups <br> <br>


Rafael Henrique Martello

_________________________________________________________________________

# [02] DATA PROCESSING

## Libraries

In [1]:
# Data handling
import pandas as pd
import numpy as np

# Graphics
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Load Data

In [2]:
# Carregando dados:

df = pd.read_csv('Data/Raw/problem_merged_data.csv')

# Data Preparation

### 1 - Null values 

In [3]:
(df.isna().sum()/df.shape[0])[(df.isna().sum()/df.shape[0])>0]

targetInterest     0.582045
targetAge          0.608799
targetOs           0.608799
targetDevices      0.608799
targetGender       0.608799
targetLanguages    0.608799
CATEGORY_1         0.380499
dtype: float64

In [4]:
# If we were working with monthly data, it would be more interesting to handle the missing values
# after splitting into train and test sets to avoid contamination.

# We have two possible approaches: remove the columns with missing values (It's not possible to remove the lines because they contain more than 60% nulls),
# or use a technique to fill (input) them (mean/median or mode imputation or methods like KNN imputer).
# in this case we will use unknown as our empty value (sometimes an empty value carries more information)

In [5]:
df = df.fillna(value="unknown")

### 2 - Identifying and removing Outliers


In [6]:

def Outliers(data, feature_list):
    '''
    Outlier detection using IQR
    '''
    # Calculate the IQR for each numeric column
    data2 = data[feature_list]
    Q1 = data2.quantile(0.025)
    Q3 = data2.quantile(0.975)
    IQR = Q3 - Q1

    # Define limits for outlier detection
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Remove outliers
    data_clean = data2[~((data2 < lower_bound) | (data2 > upper_bound)).any(axis=1)]

    # Check the size of the data before and after removing outliers
    print("Original DataFrame size:", data2.shape)
    print("DataFrame size after removing outliers:", data_clean.shape)

    if data2.shape[0] != data_clean.shape[0]:
        return data.iloc[data2.index,:]

# Example usage
df_trated = Outliers(df, list(df.select_dtypes('number').columns))




Original DataFrame size: (1682, 11)
DataFrame size after removing outliers: (1668, 11)


### 3 - Encoding categorical variables

In [7]:
# I am choosing to remove these variables at the moment
# With proper treatment, they could improve the model
# but for the current test, I am opting to exclude them


df_trated.drop(['date', 'headline', 'storySummary', 'targetGeo', 'targetInterest'], inplace=True, axis=1)

In [8]:
df_trated.to_parquet('Data/Processed/data_trated.parquet')

In [9]:
list_vars = ['group',
            'channel',
            'IABCategory',
            'targetAge',
            'targetOs',
            'targetDevices',
            'targetGender',
            'targetLanguages',
            'CATEGORY_1']

In [10]:
categorical_columns = df_trated[list_vars].select_dtypes(include=['object']).columns.tolist()
encoder = OneHotEncoder(sparse_output=False, drop='if_binary')
one_hot_encoded = encoder.fit_transform(df[categorical_columns])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))
df_encoded = pd.concat([df, one_hot_df], axis=1)
df_encoded = df_encoded.drop(categorical_columns, axis=1)

In [11]:
df_encoded.shape

(1682, 44)

In [12]:
df_encoded.select_dtypes('object').nunique().index

Index(['date', 'headline', 'storySummary', 'targetGeo', 'targetInterest'], dtype='object')

In [13]:
df_encoded.columns

Index(['group', 'item', 'date', 'bid', 'budget', 'engagements', 'page_views',
       'clicks', 'active_days', 'media_spend', 'media_cpc', 'cpe', 'headline',
       'storySummary', 'targetGeo', 'targetInterest', 'channel_MGID',
       'channel_REVCONTENT', 'channel_YAHOO',
       'IABCategory_Business and Finance', 'IABCategory_Medical Health',
       'IABCategory_News and Politics', 'IABCategory_Travel',
       'targetAge_35-44;', 'targetAge_45-54;', 'targetAge_55-64;',
       'targetAge_65+;', 'targetAge_unknown', 'targetOs_ANDROID;',
       'targetOs_IOS;', 'targetOs_unknown', 'targetDevices_unknown',
       'targetGender_FEMALE;', 'targetGender_MALE;', 'targetGender_unknown',
       'targetLanguages_unknown',
       'CATEGORY_1_/Autos & Vehicles/Vehicle Codes & Driving Laws',
       'CATEGORY_1_/Business & Industrial',
       'CATEGORY_1_/Business & Industrial/Construction & Maintenance',
       'CATEGORY_1_/Computers & Electronics/Computer Security',
       'CATEGORY_1_/Finance/Ins

In [14]:
df_encoded.shape

(1682, 44)

### 4 - Train and test split

In [15]:
train, test = train_test_split(df_encoded, test_size=0.33, random_state=42)

In [16]:
train.shape, test.shape

((1126, 44), (556, 44))

In [17]:
# I will leave the data normalization for the model pipeline

train.to_parquet('Data/Processed/train_data.parquet')
test.to_parquet('Data/Processed/test_data.parquet')